Packages

In [18]:
%pip install pandas
%pip install glob
%pip install geopandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob


In [20]:
import pandas as pd
import glob
import geopandas as gpd

Importing datasets

In [71]:
# List of files
data_folder = 'Data'
csv_files = glob.glob(f"{data_folder}/*.csv")

# Read each CSV file into a DataFrame and store in a list
dataframes = [pd.read_csv(file) for file in csv_files]

#IICA files
csv_iica = glob.glob(f"{data_folder}/*IICA/*.csv")
dataframes_iica = [pd.read_csv(file) for file in csv_iica]

#Fiscal performance files
csv_fisc = glob.glob(f"{data_folder}/*Terridata/*.csv")
dataframes_fisc = [pd.read_csv(file) for file in csv_fisc]


Cleaning datasets

In [72]:
#####################
### IICA datasets ###
#####################

### IICA 2002-2013 ###
dataframes_iica[0] = dataframes_iica[0].iloc[:, :5]
dataframes_iica[0] = dataframes_iica[0].rename(columns={
    'divipola': 'Divipola',
    'Departamento': 'Depto',
    'Municipio': 'Municipio',
    'año': 'year',
    'Índice de indicencia del conflicto': 'iica'
})

dataframes_iica[0]

,Divipola,Depto,Municipio,year,iica
0,5001,ANTIOQUIA,MEDELLÍN,2013,0.008261
1,5002,ANTIOQUIA,ABEJORRAL,2013,0.005499
2,5004,ANTIOQUIA,ABRIAQUÍ,2013,0.000327
3,5021,ANTIOQUIA,ALEJANDRÍA,2013,0.005062
4,5030,ANTIOQUIA,AMAGÁ,2013,0.016015
...,...,...,...,...,...
13459,97889,VAUPÉS,YAVARATÉ,2002,0.000185
13460,99001,VICHADA,PUERTO CARREÑO,2002,0.021377
13461,99524,VICHADA,LA PRIMAVERA,2002,0.010955
13462,99624,VICHADA,SANTA ROSALÍA,2002,0.012447


In [73]:
#####################
### IICA datasets ###
#####################

### IICA 2014-2016 ###
dataframes_iica[1] = pd.melt(dataframes_iica[1], id_vars=dataframes_iica[1].columns[:3], value_vars=dataframes_iica[1].columns[3:5], 
                    var_name='year', value_name='iica')

# Extract only the year number from the 'year' column
dataframes_iica[1]['year'] = dataframes_iica[1]['year'].str.extract('(\d+)').astype(int)

dataframes_iica[1]



<>:10: SyntaxWarning: invalid escape sequence '\d'
<>:10: SyntaxWarning: invalid escape sequence '\d'
C:\Users\cpedr\AppData\Local\Temp\ipykernel_26700\3086565869.py:10: SyntaxWarning: invalid escape sequence '\d'
  dataframes_iica[1]['year'] = dataframes_iica[1]['year'].str.extract('(\d+)').astype(int)


,Divipola,Depto,Municipio,year,iica
0,5001,ANTIOQUIA,MEDELLIN,2014,0.00692
1,5002,ANTIOQUIA,ABEJORRAL,2014,0.00738
2,5004,ANTIOQUIA,ABRIAQUI,2014,0.00070
3,5021,ANTIOQUIA,ALEJANDRIA,2014,0.00056
4,5030,ANTIOQUIA,AMAGA,2014,0.00925
...,...,...,...,...,...
2239,97889,VAUPÉS,YAVARATE,2015,0.00000
2240,99001,VICHADA,PUERTO CARREÑO,2015,0.00620
2241,99524,VICHADA,LA PRIMAVERA,2015,0.01250
2242,99624,VICHADA,SANTA ROSALIA,2015,0.00076


In [74]:
### IICA 2017-2022 ###
dataframes_iica[2] = pd.melt(dataframes_iica[2], id_vars=dataframes_iica[2].columns[1:4], value_vars=dataframes_iica[2].columns[4:10], 
                    var_name='year', value_name='iica')

# Extract only the year number from the 'year' column
dataframes_iica[2]['year'] = dataframes_iica[2]['year'].str.extract('(\d+)').astype(int)

dataframes_iica[2]

<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\d'
C:\Users\cpedr\AppData\Local\Temp\ipykernel_26700\3795539174.py:6: SyntaxWarning: invalid escape sequence '\d'
  dataframes_iica[2]['year'] = dataframes_iica[2]['year'].str.extract('(\d+)').astype(int)


,Divipola,Municipio,Depto,year,iica
0,5001,MEDELLIN,ANTIOQUIA,2017,0.02816
1,5002,ABEJORRAL,ANTIOQUIA,2017,0.00960
2,5004,ABRIAQUI,ANTIOQUIA,2017,0.03638
3,5021,ALEJANDRIA,ANTIOQUIA,2017,0.03846
4,5030,AMAGA,ANTIOQUIA,2017,0.03954
...,...,...,...,...,...
6721,97889,YAVARATE,VAUPÉS,2022,0.00000
6722,99001,PUERTO CARREÑO,VICHADA,2022,0.06498
6723,99524,LA PRIMAVERA,VICHADA,2022,0.03024
6724,99624,SANTA ROSALIA,VICHADA,2022,0.00344


In [ ]:
# Merge all IICA datasets
merged_iica = pd.concat(dataframes_iica, ignore_index=True)

# Display the merged dataframe
merged_iica.describe()

,Divipola,year,iica
count,22434.000000,22434.000000,22434.000000
mean,38701.447892,2011.797941,0.031321
std,26551.439316,6.136376,0.077141
min,5001.000000,2002.000000,0.000000
25%,15676.000000,2006.000000,0.002523
50%,25839.000000,2011.000000,0.009197
75%,66572.000000,2017.000000,0.026776
max,99773.000000,2022.000000,1.000000


In [ ]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# Assuming you have two DataFrames: df1 (2005-2009) and df2 (2011-2022)
# Combine the two DataFrames
combined_df = pd.concat([df1, df2])

# Set 'year' as index for interpolation and extrapolation
combined_df.set_index('year', inplace=True)

# Interpolate linearly to fill in 2010
combined_df = combined_df.reindex(range(2005, 2023))  # include 2010 in index range
combined_df['GDP'] = combined_df['GDP'].interpolate(method='linear')

# Prepare data for polynomial regression
years = combined_df.dropna().index.values.reshape(-1, 1)
gdp = combined_df.dropna()['GDP'].values

# Fit polynomial regression model
poly = PolynomialFeatures(degree=2)
years_poly = poly.fit_transform(years)
model = LinearRegression().fit(years_poly, gdp)

# Predict GDP for years before 2005
years_before_2005 = np.arange(2000, 2005).reshape(-1, 1)
years_before_2005_poly = poly.transform(years_before_2005)
gdp_before_2005 = model.predict(years_before_2005_poly)

# Add predicted values to the DataFrame
for year, gdp_value in zip(years_before_2005.flatten(), gdp_before_2005):
    combined_df.loc[year] = gdp_value

# Sort the DataFrame by year
combined_df.sort_index(inplace=True)

print(combined_df)

In [ ]:


# Read the shapefile
shapes = gpd.read_file('Data/shapes/shapes.shp')

shapes

,area,perimeter,dpto,nombre_dpt,mpio,nombre_mpi,nombre_cab,mpios,hectares,geometry
0,1.775410e+09,313036.280961,05,ANTIOQUIA,873,VIGIA DEL FUERTE,VIGIA DEL FUERTE,05873,177540.955,"POLYGON ((-76.90331 6.83561, -76.90562 6.82957..."
1,8.723712e+08,151181.011238,05,ANTIOQUIA,686,SANTA ROSA DE OSOS,SANTA ROSA DE OSOS,05686,87237.121,"POLYGON ((-75.6099 6.8743, -75.6071 6.8708, -7..."
2,1.317151e+09,178275.152241,05,ANTIOQUIA,284,FRONTINO,FRONTINO,05284,131715.082,"POLYGON ((-76.2441 6.9174, -76.2227 6.9167, -7..."
3,8.223566e+07,44796.540606,05,ANTIOQUIA,315,GUADALUPE,GUADALUPE,05315,8223.566,"POLYGON ((-75.2312 6.9374, -75.2232 6.9287, -7..."
4,3.959024e+08,113140.065596,68,SANTANDER,895,ZAPATOCA,ZAPATOCA,68895,39590.242,"POLYGON ((-73.2425 6.9088, -73.2309 6.9047, -7..."
...,...,...,...,...,...,...,...,...,...,...
1117,1.110146e+08,55343.749206,47,MAGDALENA,205,CONCORDIA,CONCORDIA,47205,11101.462,"POLYGON ((-74.69577 10.22744, -74.69933 10.226..."
1118,3.415701e+08,105759.091575,47,MAGDALENA,960,ZAPAYAN,ZAPAYAN,47960,34157.008,"POLYGON ((-74.70518 10.27644, -74.69872 10.263..."
1119,5.837940e+08,151715.069375,17,CALDAS,380,LA DORADA,LA DORADA,17380,58379.399,"POLYGON ((-74.6766 5.7584, -74.6754 5.7463, -7..."
1120,2.816016e+09,359760.445480,27,CHOCO,150,CARMEN DEL DARIEN,CARMEN DEL DARIEN,27150,281601.625,"POLYGON ((-76.89271 7.24821, -76.89113 7.22927..."
